Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)

3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (10 баллов)

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (35 баллов)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.

In [368]:
import pandas as pd
from calendar import monthrange
import datetime
from operator import attrgetter

In [134]:
df_customers = pd.read_csv('./data/unique_id.csv')
df_orders = pd.read_csv('./data/order_list.csv')
df_items = pd.read_csv('./data/order_items.csv')

Сколько у нас пользователей, которые совершили покупку только один раз?

In [45]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


Проверяем какие есть статусы заказов

In [46]:
df_orders['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

Оставляем только завершенные заказы со статусом delivered и считаем количество пользователей с одним заказом

In [47]:
df_delivered = df_orders[df_orders['order_status']=='delivered']

In [ ]:
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [49]:
df_united = df_customers.merge(df_delivered, on='customer_id')

In [50]:
df_united_grouped = df_united[['customer_unique_id','order_id']].groupby('customer_unique_id').count().reset_index()

In [52]:
customers_delivered = df_united_grouped[df_united_grouped['order_id']==1].shape[0]

In [53]:
customers_delivered

90557

In [58]:
print(f'Количество пользователей с одним заказом = {customers_delivered}')

Количество пользователей с одним заказом = 90557


### Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)

In [64]:
df_undelivered = df_orders[df_orders['order_status'].isin(['canceled','unavailable'])]

In [84]:
df_undelivered.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaN,NaN,2017-12-05 00:00:00
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaN,NaN,2018-08-14 00:00:00
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaN,NaN,2018-02-16 00:00:00
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaN,NaN,2017-09-05 00:00:00


In [97]:
df_undelivered['order_purchase_timestamp'] = pd.to_datetime(df_undelivered.order_purchase_timestamp)

/var/folders/58/sd6nz3qn3xb1v_qsf3_25dv80000gn/T/ipykernel_9340/4108323369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_undelivered['order_purchase_timestamp'] = pd.to_datetime(df_undelivered.order_purchase_timestamp)


In [105]:
df_undelivered['order_purchase_timestamp'].dt.date().replace(day=1)

TypeError: 'Series' object is not callable

In [114]:
df_undelivered['order_date'] = df_undelivered['order_purchase_timestamp'].apply(lambda x: x.date().replace(day=1))

/var/folders/58/sd6nz3qn3xb1v_qsf3_25dv80000gn/T/ipykernel_9340/3105580286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_undelivered['order_date'] = df_undelivered['order_purchase_timestamp'].apply(lambda x: x.date().replace(day=1))


In [115]:
df_undelivered.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,data_order_purchase,order_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaN,NaN,2017-12-05 00:00:00,2017-11-16 15:09:28,2017-11-01
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaN,NaN,2018-08-14 00:00:00,2018-08-04 14:29:27,2018-08-01
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaN,NaN,2018-02-16 00:00:00,2018-01-31 11:31:37,2018-01-01
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00,2018-01-26 21:34:08,2018-01-01
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaN,NaN,2017-09-05 00:00:00,2017-08-14 17:38:02,2017-08-01


In [128]:
df_grouped_month = df_undelivered[['order_date','order_status','order_id']]\
                .groupby(['order_date','order_status'])\
                .count().reset_index()\
                .rename(columns={'order_id':'count'})

In [129]:
df_grouped_month.head()

,order_date,order_status,count
0,2016-09-01,canceled,2
1,2016-10-01,canceled,24
2,2016-10-01,unavailable,7
3,2017-01-01,canceled,3
4,2017-01-01,unavailable,10


In [133]:
df_grouped_month.groupby(['order_status']).mean().reset_index()

,order_status,count
0,canceled,26.041667
1,unavailable,29.000000


### По каждому товару определить, в какой день недели товар чаще всего покупается.

In [135]:
df_orders_items = df_orders.merge(df_items, on = 'order_id')

In [136]:
df_orders_items.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [142]:
df_orders_items['order_purchase_timestamp']= pd.to_datetime(df_orders_items.order_purchase_timestamp)

In [148]:
df_orders_items['order_weekday'] = df_orders_items['order_purchase_timestamp'].apply(lambda x: x.date().strftime("%A"))

In [149]:
df_orders_items.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_weekday
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,Monday
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,Tuesday
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,Wednesday
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,Saturday
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,Tuesday


In [158]:
df_orders_items_weekday = df_orders_items[['product_id','order_weekday','order_id']]\
                .groupby(['product_id', 'order_weekday'])\
                .count()\
                .reset_index()\
                .rename(columns={'order_id':'count'})

In [161]:
df_orders_items_weekday.head()

,product_id,order_weekday,count
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1
2,0009406fd7479715e4bef61dd91f2462,Thursday,1
3,000b8f95fcb9e0096488278317764d19,Friday,1
4,000b8f95fcb9e0096488278317764d19,Wednesday,1


In [168]:
df_orders_items_weekday_max = df_orders_items_weekday[['product_id','count']].groupby(['product_id'])\
                        .max()\
                        .reset_index()

In [171]:
df_orders_items_weekday_max.merge(df_orders_items_weekday, on = ['product_id','count'])

,product_id,count,order_weekday
0,00066f42aeeb9f3007548bb9d3f33c38,1,Sunday
1,00088930e925c41fd95ebfe695fd2655,1,Tuesday
2,0009406fd7479715e4bef61dd91f2462,1,Thursday
3,000b8f95fcb9e0096488278317764d19,1,Friday
4,000b8f95fcb9e0096488278317764d19,1,Wednesday
...,...,...,...
42700,fff6177642830a9a94a0f2cba5e476d1,1,Sunday
42701,fff81cc3158d2725c0655ab9ba0f712c,1,Monday
42702,fff9553ac224cec9d15d49f5a263411f,1,Friday
42703,fffdb2d0ec8d6a61f0a0a0db3f25b441,2,Tuesday


### Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть

In [173]:
df_customers_orders = df_customers.merge(df_orders, on = ['customer_id'])

In [174]:
df_customers_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


1. Достать из даты дату начала каждого месяца 
2. Посчитать количество недель в каждом месяце
3. Посчитать количество покупок каждого пользователя в каждом месяце
4. Разделить количество покупок каждого пользователя в каждом месяце на количество недель в каждом месяце 


In [176]:
df_customers_orders['order_purchase_timestamp'] = pd.to_datetime(df_customers_orders.order_purchase_timestamp)

In [177]:
df_customers_orders['order_month'] = df_customers_orders['order_purchase_timestamp'].apply(lambda x: x.date().replace(day=1))

In [178]:
df_customers_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_month
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05-01
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,2018-01-01
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,2018-05-01
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,2018-03-01
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,2018-07-01


In [186]:
df_customers_orders_count = df_customers_orders[['customer_unique_id','order_month','order_id']].groupby(['customer_unique_id','order_month'])\
                    .count()\
                    .reset_index()

In [187]:
df_customers_orders_count.head()

,customer_unique_id,order_month,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01,1
2,0000f46a3911fa3c0805444483337064,2017-03-01,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01,1
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01,1


In [190]:
monthrange(2021,11)

(0, 30)

In [198]:
ya_object = df_customers_orders['order_month'][0]
monthrange(ya_object.year, ya_object.month)[1]

31

In [205]:
df_customers_orders_count['days_in_month'] = df_customers_orders_count['order_month'].apply(lambda x: monthrange(x.year, x.month)[1])

In [206]:
df_customers_orders_count['weeks_in_month'] = df_customers_orders_count['days_in_month']/7

In [207]:
df_customers_orders_count.head()

,customer_unique_id,order_month,order_id,days_in_month,weeks_in_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01,1,31,4.428571
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01,1,31,4.428571
2,0000f46a3911fa3c0805444483337064,2017-03-01,1,31,4.428571
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01,1,31,4.428571
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01,1,30,4.285714


In [210]:
df_customers_orders_count = df_customers_orders_count.rename(columns={'order_id':'count_orders'}) 

In [211]:
df_customers_orders_count['avg_orders'] = df_customers_orders_count['count_orders']/df_customers_orders_count['weeks_in_month']

In [212]:
df_customers_orders_count.head()

,customer_unique_id,order_month,count_orders,days_in_month,weeks_in_month,avg_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01,1,31,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01,1,31,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03-01,1,31,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01,1,31,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01,1,30,4.285714,0.233333


### Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.  

1. Взять предыдущую таблицу
2. Найти дату первой покупки в каждом месяце
3. Найти кагорты в период с января по март, с февраля по апрель, с марта по май, с апреля по июнь, с мая по июль, с июня по август, с июля по сентябрь, с августа по октябрь, с сентября по ноябрь, с октября по декабрь 




In [249]:
df_customers_orders_count.sort_values('order_month') 

,customer_unique_id,order_month,count_orders,days_in_month,weeks_in_month,avg_orders,january
70504,b7d76e111c89f7ebf14761390f0f7d17,2016-09-01,1,30,4.285714,0.233333,2016-09-01
27787,4854e9b3feff728c13ee5fc7d1547e92,2016-09-01,1,30,4.285714,0.233333,2016-09-01
50307,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-01,1,30,4.285714,0.233333,2016-09-01
242,009b0127b727ab0ba422f6d9604487c7,2016-09-01,1,30,4.285714,0.233333,2016-09-01
79611,cfbb7c1c3d613d974eef515b180e2c9b,2016-10-01,1,31,4.428571,0.225806,2016-10-01
...,...,...,...,...,...,...,...
57680,968fac81e2c44fb6c1e3ac2a45e6a102,2018-09-01,1,30,4.285714,0.233333,2018-09-01
38121,634420a0ea42302205032ed44ac7fccc,2018-10-01,1,31,4.428571,0.225806,2018-10-01
67306,af5454198a97379394cacf676e1e96cb,2018-10-01,1,31,4.428571,0.225806,2018-10-01
14656,262e1f1e26e92e86375f86840b4ffd63,2018-10-01,1,31,4.428571,0.225806,2018-10-01


In [286]:
customers_orders_count_month = df_customers_orders_count[['customer_unique_id', 'order_month']].groupby('customer_unique_id').min().reset_index()
                     
                                                                

In [311]:
customers_orders_count_month.head()

,customer_unique_id,order_month
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01
2,0000f46a3911fa3c0805444483337064,2017-03-01
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01


In [458]:
customers_orders_count__jan = customers_orders_count_month[customers_orders_count_month['order_month'] == datetime.date(2018, 1, 1)]

In [459]:
customers_orders_count__jan.head()

,customer_unique_id,order_month
17,000d460961d6dbfa3ec6c9f5805769e1,2018-01-01
31,0014a5a58da615f7b01a4f5e194bf5ea,2018-01-01
56,0028cfeb9dfcd3628a56d5b6400deef5,2018-01-01
64,002d3bd901608f67c3fc11eaaa842b13,2018-01-01
68,002ed12115742033f015cb3c269ccf68,2018-01-01


In [460]:
df_all_orders = df_customers_orders_count[['customer_unique_id','order_month']].drop_duplicates()

In [461]:
df_customers_orders_count[['customer_unique_id','order_month']].shape

(98046, 2)

In [462]:
customers_orders_count__jan = customers_orders_count__jan.merge(df_all_orders, on = 'customer_unique_id')

In [467]:
customers_orders_count__jan.head()

,customer_unique_id,order_month_x,order_month_y
0,000d460961d6dbfa3ec6c9f5805769e1,2018-01-01,2018-01-01
1,0014a5a58da615f7b01a4f5e194bf5ea,2018-01-01,2018-01-01
2,0028cfeb9dfcd3628a56d5b6400deef5,2018-01-01,2018-01-01
3,002d3bd901608f67c3fc11eaaa842b13,2018-01-01,2018-01-01
4,002ed12115742033f015cb3c269ccf68,2018-01-01,2018-01-01


In [468]:
customers_orders_count__jan[customers_orders_count__jan['order_month_y'] == datetime.date(2018, 3, 1)].shape

(27, 3)

In [469]:
customers_orders_count__jan.shape

(7155, 3)

In [470]:
df_cohort = customers_orders_count__jan.groupby('order_month_y').count().reset_index()

In [471]:
df_cohort.head()

,order_month_y,customer_unique_id,order_month_x
0,2018-01-01,7025,7025
1,2018-02-01,24,24
2,2018-03-01,27,27
3,2018-04-01,20,20
4,2018-05-01,20,20


In [472]:
df_cohort['cohort_month'] = datetime.date(2018, 1, 1)

In [473]:
df_cohort.head()

,order_month_y,customer_unique_id,order_month_x,cohort_month
0,2018-01-01,7025,7025,2018-01-01
1,2018-02-01,24,24,2018-01-01
2,2018-03-01,27,27,2018-01-01
3,2018-04-01,20,20,2018-01-01
4,2018-05-01,20,20,2018-01-01


In [445]:
# df_cohort_pivoted = df_cohort.pivot_table(index = 'cohort_month',
#                                      columns = 'order_month_y',
#                                      values = 'customer_unique_id')#.reset_index()

In [447]:
cohort_size

cohort_month
2018-01-01    7025
Name: 2018-01-01, dtype: int64

In [448]:
df_cohort_pivoted

order_month_y,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01
cohort_month,,,,,,,,
2018-01-01,7025,24,27,20,20,11,12,16


In [476]:
df_cohort['period'] = df_cohort.apply(lambda x: x.order_month_y.month - x.cohort_month.month, axis=1)

In [477]:
df_cohort.head()

,order_month_y,customer_unique_id,order_month_x,cohort_month,period
0,2018-01-01,7025,7025,2018-01-01,0
1,2018-02-01,24,24,2018-01-01,1
2,2018-03-01,27,27,2018-01-01,2
3,2018-04-01,20,20,2018-01-01,3
4,2018-05-01,20,20,2018-01-01,4


In [478]:
df_cohort_pivoted = df_cohort.pivot_table(index = 'order_mcohort_monthonth_y',
                                     columns = 'period',
                                     values = 'customer_unique_id')#.reset_index()

In [483]:
df_cohort_pivoted

period,0,1,2,3,4,5,6,7
cohort_month,,,,,,,,
2018-01-01,7025,24,27,20,20,11,12,16


In [484]:
cohort_size = df_cohort_pivoted.iloc[:,0]
df_cohort_pivoted.divide(cohort_size, axis = 0)

period,0,1,2,3,4,5,6,7
cohort_month,,,,,,,,
2018-01-01,1.0,0.003416,0.003843,0.002847,0.002847,0.001566,0.001708,0.002278
